In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("train.csv")

In [4]:
df.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


In [5]:
# Define a function to calculate mean absolute error
def calculate_mae(y):
    return sum(abs(y - y.mean())) / len(y)

# Define a function to split the dataset based on a feature and value
def split_dataset(X, y, value, column):
    left_mask = X[column] <= value
    right_mask = X[column] > value
    return X[left_mask], X[right_mask], y[left_mask], y[right_mask]

# Define a function to find the best split in the dataset
def find_best_split(X, y):
    best_mae = float('inf')
    best_value, best_column = None, None

    for column in X.columns:
        for value in set(X[column]):
            left_X, right_X, left_y, right_y = split_dataset(X, y, value, column)
            current_mae = calculate_mae(left_y) + calculate_mae(right_y)

            if current_mae < best_mae:
                best_mae = current_mae
                best_value, best_column = value, column

    return best_value, best_column

# Define a recursive function to build the decision tree
def build_tree(X, y, depth=1, max_depth=3):
    if depth > max_depth or len(set(y)) == 1:
        return {'prediction': y.mean()}

    value, column = find_best_split(X, y)

    if value is None or column is None:
        return {'prediction': y.mean()}

    left_X, right_X, left_y, right_y = split_dataset(X, y, value, column)

    tree = {
        'value': value,
        'column': column,
        'left': build_tree(left_X, left_y, depth + 1, max_depth),
        'right': build_tree(right_X, right_y, depth + 1, max_depth)
    }

    return tree

# Train the decision tree on your dataset
X_train = df[['Feature1', 'Feature2']]
y_train = df['Target']
tree_model = build_tree(X_train, y_train)

# Define a function to make predictions with the decision tree
def predict(tree, row):
    if 'prediction' in tree:
        return tree['prediction']

    if row[tree['column']] <= tree['value']:
        return predict(tree['left'], row)
    else:
        return predict(tree['right'], row)

# Make predictions on your training data
predictions_train = X_train.apply(lambda row: predict(tree_model, row), axis=1)

# Evaluate the model performance on the training data
mae_train = calculate_mae(predictions_train)
print(f'Training Mean Absolute Error: {mae_train}')


KeyError: "None of [Index(['Feature1', 'Feature2'], dtype='object')] are in the [columns]"